In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18258809
paper_name = 'thomsen_jensen_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/hits.txt', header=None, sep='\t')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 15 x 2


In [7]:
original_data.head()

,0,1
0,THO2,++
1,THP2,+++
2,MFT1,+++++
3,HPR1,++++
4,SAC3,++++


In [15]:
original_data['gene'] = original_data[0].astype(str)

In [16]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [20]:
original_data.loc[original_data['gene']=='RIP1NUP42','gene'] = 'NUP42'

In [21]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [22]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, gene, orf]
Index: []


In [23]:
original_data['data'] = original_data[1].apply(lambda x: -len(x))

In [24]:
original_data.set_index('orf', inplace=True)

In [25]:
original_data = original_data[['data']].copy()

In [26]:
original_data = original_data.groupby(original_data.index).mean()

In [27]:
original_data.shape

(15, 1)

# Load & process tested strains

In [28]:
tested = pd.read_excel('raw_data/Euroscarf_THJlab.xlsx', sheet_name='INDEX')

In [29]:
tested.head()

,Unnamed: 0,record #,gene,alt. gene,syst. gene,strain,batch,plate?? please check in the folder!!!,row,col,comments,slow growth?
0,1st delivery,338,PAU8,NaN,YAL068C,BY4741,chr1_1,1,A,2,NaN,NaN
1,1st delivery,339,SEO1,NaN,YAL067C,BY4741,chr1_1,1,A,3,NaN,NaN
2,1st delivery,340,NaN,NaN,YAL066W,BY4741,chr1_1,1,A,4,NaN,NaN
3,1st delivery,341,NaN,NaN,YAL065C,BY4741,chr1_1,1,A,5,NaN,NaN
4,1st delivery,345,GDH3,FUN51,YAL062W,BY4741,chr1_1,1,A,9,NaN,NaN


In [30]:
tested['orf'] = tested['syst. gene'].astype(str)

In [31]:
tested['orf'] = clean_orf(tested['orf'])

In [32]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [33]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, record #, gene, alt. gene, syst. gene, strain, batch, plate?? please check in the folder!!!, row, col, comments, slow growth?, orf]
Index: []


In [34]:
tested_orfs = tested['orf'].unique()

In [35]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [36]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [37]:
data = original_data.copy()

In [38]:
dataset_ids = [107]
datasets = datasets.reindex(index=dataset_ids)

In [39]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [40]:
data.head()

dataset_id,107
data_type,value
orf,
YAL068C,0
YAL067C,0
YAL066W,0
YAL065C,0
YAL062W,0


## Subset to the genes currently in SGD

In [41]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [42]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,107
,data_type,value
gene_id,orf,
1869,YAL068C,0
61,YAL067C,0
60,YAL066W,0
1727,YAL065C,0
57,YAL062W,0


# Normalize

In [43]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [44]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [45]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        107       
data_type       value valuez
gene_id orf                 
1869    YAL068C     0    0.0
61      YAL067C     0    0.0
60      YAL066W     0    0.0
1727    YAL065C     0    0.0
57      YAL062W     0    0.0

# Print out

In [46]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [47]:
from IO.save_data_to_db3 import *

In [48]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18258809...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.87s/it]

Updating the data_modified_on field...
